In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_wine, load_boston
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split



In [37]:
import numpy as np
from collections import Counter
import random
from tqdm import tqdm


class MyDecisionTreeClassifier:

    def __init__(self, max_depth=None, max_features=2, min_leaf_samples=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_leaf_samples = min_leaf_samples
        self._node = {
                        'left': None,
                        'right': None,
                        'feature': None,
                        'threshold': None,
                        'depth': 0,
                        'classes_proba': None
                    }
        self.tree = None  # словарь в котором будет храниться построенное дерево
        self.classes = None  # список меток классов

    def fit(self, X, y):
        self.classes = np.unique(y)  
        self.tree = {'root': self._node.copy()}  # создаём первую узел в дереве
        self._build_tree(self.tree['root'], X, y)  # запускаем рекурсивную функцию для построения дерева
        return self

    def predict_proba(self, X):
        proba_preds = []
        for x in X:
            preds_for_x = self._get_predict(self.tree['root'], x)  # рекурсивно ищем лист в дереве соответствующий объекту
            proba_preds.append(preds_for_x)
        return np.array(proba_preds)

    def predict(self, X):
        proba_preds = self.predict_proba(X)
        preds = proba_preds.argmax(axis=1).reshape(-1, 1)
        return preds

    def get_best_split(self, X, y):

        best_Q = None
        best_j = None
        best_t = None
        best_left_ids = None
        best_right_ids = None
        
        full_gini = self.gini(y)
        features = self.get_random_features_inds(X.shape[1])
        for j in features:
            split_values = self.find_splits(X, j)
            for t in split_values:
                left_mask = X[:, j] <= t
                right_mask = X[:, j] > t
                left_y, right_y = y[left_mask], y[right_mask]
                Q_cur = self.calc_Q(left_y, right_y, full_gini)
                if best_Q is None or Q_cur > best_Q:
                    best_Q = Q_cur
                    best_j = j
                    best_t = t
                    best_left_ids = left_mask
                    best_right_ids = right_mask

        return best_j, best_t, best_left_ids, best_right_ids

    def calc_Q(self, y_left, y_right, full_gini):
        n_l = len(y_left)
        n_r = len(y_right)
        n = n_l + n_r
        return full_gini - (n_l*self.gini(y_left) + n_r*self.gini(y_right))/n

    def gini(self, y):
        n = len(y)
        if n < 2:
            return 0
        y=np.array(y).reshape(1, -1).squeeze()
        counter = Counter(y)
        arr = []
        for c in counter:
            arr.append((counter[c]/n)**2)
        arr = np.array(arr)
        return 1 - np.sum(arr)
    
    def find_splits(self, X, column):
        X_unique = np.unique(X[:, column])
        split_values = np.empty(X_unique.shape[0] - 1)
        for i in range(1, X_unique.shape[0]):
            average = (X_unique[i - 1] + X_unique[i]) / 2
            split_values[i - 1] = average
        return split_values
    
    def get_random_features_inds(self, n):
        all_inds = np.arange(n)
        random.shuffle(all_inds)
        return all_inds[:self.max_features]
        
        

    def _build_tree(self, curr_node, X, y):

        if curr_node['depth'] == self.max_depth:  # выход из рекурсии если построили до максимальной глубины
            # сохраняем предсказания листьев дерева перед выходом из рекурсии
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}
            return

        if len(np.unique(y)) == 1:  # выход из рекурсии значения если "y" одинковы для все объектов
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}
            return

        j, t, left_ids, right_ids = self.get_best_split(X, y)  # нахождение лучшего разбиения

        curr_node['feature'] = j  # признак по которому производится разбиение в текущем узле
        curr_node['threshold'] = t  # порог по которому производится разбиение в текущем узле

        left = self._node.copy()  # создаём узел для левого поддерева
        right = self._node.copy()  # создаём узел для правого поддерева

        left['depth'] = curr_node['depth'] + 1  # увеличиваем значение глубины в узлах поддеревьев
        right['depth'] = curr_node['depth'] + 1

        curr_node['left'] = left
        curr_node['right'] = right

        self._build_tree(left, X[left_ids], y[left_ids])  # продолжаем построение дерева
        self._build_tree(right, X[right_ids], y[right_ids])

    def _get_predict(self, node, x):
        if node['threshold'] is None:  # если в узле нет порога, значит это лист, выходим из рекурсии
            return [node['classes_proba'][c] for c in self.classes]

        if x[node['feature']] <= node['threshold']:  # уходим в правое или левое поддерево в зависимости от порога и признака
            return self._get_predict(node['left'], x)
        else:
            return self._get_predict(node['right'], x)


def read_matrix(n, dtype=float):
    matrix = np.array([list(map(dtype, input().split())) for _ in range(n)])
    return matrix

def read_input_matriсes(n, m, k):
    X_train, y_train, X_test = read_matrix(n), read_matrix(n), read_matrix(k)
    return X_train, y_train, X_test

def print_matrix(matrix):
    for row in matrix:
        print(' '.join(map(str, row)))

def solution():
    n, m, k = 3, 2, 2
    X_train = np.array([[0.0, 1.0],
                        [0.0, 2.0], 
                        [0.0, 3.0]])
    y_train = np.array([[1.], [2.], [3.]]) 
    X_test = np.array([[0.0, 1.0], [0.0, 2.0]])

    clf = MyDecisionTreeClassifier()
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    proba_preds = clf.predict_proba(X_test).round(4)

    print_matrix(preds)
    print_matrix(proba_preds)


solution()

0
1
1.0 0.0 0.0
0.0 1.0 0.0


### Датасет "Wine recognition"

In [28]:
wine_dataset = load_wine()
data = wine_dataset['data']
target = wine_dataset['target']

In [27]:
print(wine_dataset['DESCR'])

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

---

In [30]:
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(data, target, random_state=2, test_size=0.33)

In [45]:
X_test_wine.shape

(59, 13)

In [31]:
tree = DecisionTreeClassifier(max_features=X_train_wine.shape[1], random_state=1)
tree.fit(X_train_wine, y_train_wine)
(y_test_wine == tree.predict(X_test_wine)).mean()

0.864406779661017

**Статья на вики про [случайный лес](https://ru.wikipedia.org/wiki/Random_forest).**

Случайный лес:

* Обучить много глубоких деревьев для решения задачи;
* Усреднить их ответы на этапе предсказания.

Задание:

    1) Реализуйте случайный лес решающих деревьев пользуясь классом DecisionTreeClassifier в качестве базового алгоритма;
    2) Обучите ваш классификатор с на данных Wine recognition dataset;
    3) Сравните полученный результат с бейзлайном на одном Decision tree;
    4) Постройте график зависимости точности от количества деревьев;
    5) Постройте график зависимости точности от параметра max_features в базовых деревьях;
    6) Постройте шрафик зависимости точности от глубины базовых алгоритмов.
    
Обратите внимание, что количество классов может быть > 2.

In [138]:
import numpy as np
from collections import Counter
import random
from tqdm import tqdm


class MyDecisionTreeClassifier:

    def __init__(self, max_depth=None, max_features=2, min_leaf_samples=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_leaf_samples = min_leaf_samples
        self._node = {
                        'left': None,
                        'right': None,
                        'feature': None,
                        'threshold': None,
                        'depth': 0,
                        'classes_proba': None
                    }
        self.tree = None  # словарь в котором будет храниться построенное дерево
        self.classes = None  # список меток классов

    def fit(self, X, y, n_classes):
        self.classes = np.arange(n_classes) 
        self.tree = {'root': self._node.copy()}  # создаём первую узел в дереве
        self._build_tree(self.tree['root'], X, y)  # запускаем рекурсивную функцию для построения дерева
        return self

    def predict_proba(self, X):
        proba_preds = []
        for x in X:
            preds_for_x = self._get_predict(self.tree['root'], x)  # рекурсивно ищем лист в дереве соответствующий объекту
            proba_preds.append(preds_for_x)
        return np.array(proba_preds)

    def predict(self, X):
        proba_preds = self.predict_proba(X)
        preds = proba_preds.argmax(axis=1).reshape(-1, 1)
        return preds

    def get_best_split(self, X, y):

        best_Q = None
        best_j = None
        best_t = None
        best_left_ids = None
        best_right_ids = None
        
        full_gini = self.gini(y)
        features = self.get_random_features_inds(X.shape[1])
        for j in features:
            split_values = self.find_splits(X, j)
            for t in split_values:
                left_mask = X[:, j] <= t
                right_mask = X[:, j] > t
                left_y, right_y = y[left_mask], y[right_mask]
                Q_cur = self.calc_Q(left_y, right_y, full_gini)
                if best_Q is None or Q_cur > best_Q:
                    best_Q = Q_cur
                    best_j = j
                    best_t = t
                    best_left_ids = left_mask
                    best_right_ids = right_mask

        return best_j, best_t, best_left_ids, best_right_ids

    def calc_Q(self, y_left, y_right, full_gini):
        n_l = len(y_left)
        n_r = len(y_right)
        n = n_l + n_r
        return full_gini - (n_l*self.gini(y_left) + n_r*self.gini(y_right))/n

    def gini(self, y):
        n = len(y)
        if n < 2:
            return 0
        y=np.array(y).reshape(1, -1).squeeze()
        counter = Counter(y)
        arr = []
        for c in counter:
            arr.append((counter[c]/n)**2)
        arr = np.array(arr)
        return 1 - np.sum(arr)
    
    def find_splits(self, X, column):
        X_unique = np.unique(X[:, column])
        split_values = np.empty(X_unique.shape[0] - 1)
        for i in range(1, X_unique.shape[0]):
            average = (X_unique[i - 1] + X_unique[i]) / 2
            split_values[i - 1] = average
        return split_values
    
    def get_random_features_inds(self, n):
        all_inds = np.arange(n)
        random.shuffle(all_inds)
        return all_inds[:self.max_features]
        
        

    def _build_tree(self, curr_node, X, y):

        if curr_node['depth'] == self.max_depth:  # выход из рекурсии если построили до максимальной глубины
            # сохраняем предсказания листьев дерева перед выходом из рекурсии
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}
            return

        if len(np.unique(y)) == 1:  # выход из рекурсии значения если "y" одинковы для все объектов
            curr_node['classes_proba'] = {c: (y == c).mean() for c in self.classes}
            return

        j, t, left_ids, right_ids = self.get_best_split(X, y)  # нахождение лучшего разбиения

        curr_node['feature'] = j  # признак по которому производится разбиение в текущем узле
        curr_node['threshold'] = t  # порог по которому производится разбиение в текущем узле

        left = self._node.copy()  # создаём узел для левого поддерева
        right = self._node.copy()  # создаём узел для правого поддерева

        left['depth'] = curr_node['depth'] + 1  # увеличиваем значение глубины в узлах поддеревьев
        right['depth'] = curr_node['depth'] + 1

        curr_node['left'] = left
        curr_node['right'] = right

        self._build_tree(left, X[left_ids], y[left_ids])  # продолжаем построение дерева
        self._build_tree(right, X[right_ids], y[right_ids])

    def _get_predict(self, node, x):
        if node['threshold'] is None:  # если в узле нет порога, значит это лист, выходим из рекурсии
            return [node['classes_proba'][c] for c in self.classes]

        if x[node['feature']] <= node['threshold']:  # уходим в правое или левое поддерево в зависимости от порога и признака
            return self._get_predict(node['left'], x)
        else:
            return self._get_predict(node['right'], x)


class MyRandomForestClassifier:
    
    def __init__(self, max_features, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.forest = []
        
    def fit(self, X, y):
        
        self.forest = []
        self.n_classes = len(np.unique(y))
        
        """You code here"""
        for _ in tqdm(range(self.n_estimators)):
            index = np.random.choice(X.shape[0], X.shape[0])
            tree = MyDecisionTreeClassifier(max_features=self.max_features)
            tree.fit(X[index], y[index], self.n_classes)
            self.forest.append(tree)
        return self
    
    def predict(self, X):
        preds = self.predict_proba(X)
        return np.argmax(preds, axis=1)
    
    def predict_proba(self, X):
        
        """You code here"""
        preds = []
        for tree in self.forest:
            preds.append(tree.predict_proba(X))
        return np.array(preds).mean(axis=0)

In [139]:
rf = MyRandomForestClassifier(max_features=4, n_estimators=105)
rf.fit(X_train_wine, y_train_wine)
(y_test_wine == rf.predict(X_test_wine)).mean()

100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [00:03<00:00, 28.07it/s]


1.0

In [147]:
def solution():
    n, m, k = 3, 4, 2
    X_train = np.array([[1.0, 0.0, 0.0, 0.0],
                        [2.0, 0.0, 0.0, 0.0],
                        [3.0, 0.0, 0.0, 0.0]]) 
    y_train =  np.array([[0],
                        [1],
                        [2]]) 
    X_test = np.array([[1.0, 0.0, 0.0, 0.0],
                       [2.0, 0.0, 0.0, 0.0]])

    rf = MyRandomForestClassifier(max_features=4, n_estimators=200)  
    rf.fit(X_train, y_train)

    predictions = rf.predict_proba(X_test)
    print_matrix(predictions)

solution()

100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 2494.36it/s]

0.665 0.3 0.035
0.21 0.755 0.035


In [47]:
X_test = np.array([[1.0, 0.0, 0.0, 0.0],
                       [2.0, 0.0, 0.0, 0.0]])
X_test.shape

(2, 4)

### Датасет "Boston house prices"

In [151]:
boston_dataset = load_boston()

data_boston = boston_dataset['data']
target_boston = boston_dataset['target']

print(boston_dataset['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [152]:
X_train_boston, X_test_boston,\
y_train_boston, y_test_boston = train_test_split(data_boston, target_boston, random_state=2, test_size=0.33)

---

**Статья на вики про [градиентный бустинг](https://en.wikipedia.org/wiki/Gradient_boosting).**

**Градиентный бустинг**

* Инициализировать ответ ансамбля $a_0$ нулями;
* Вычислить градиент функционала потерь $L(y - a_0)$;
* Обучить базовый $b_1$ регрессор предсказывать антиградиент $-L(y - a_0)$;
* Добавить базовый алгоритм $b_1$ с весом $\eta$ к композиции;
* Повторять пока ошибка уменьшается на валидации;

Задание:

    1) Реализуйте градиентный бустинг над решающими деревьями пользуясь классом DecisionTreeRegressor в качестве базового алгоритма;
    2) Обучите ваш регрессор на данных Boston house prices;
    3) Сравните полученный результат по метрике MAPE с RandomForestRegressor(n_estimators=2000) из sklearn;
    4) Попробуйте подобрать такие learning_rate и n_estimators для градиентного бустинга, чтобы ошибка MAPE была меньше 10%.

In [422]:
import numpy as np
from collections import Counter
import random
from tqdm import tqdm


class MyDecisionTreeRegressor:

    def __init__(self, max_depth=None, max_features=None, min_leaf_samples=None):
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_leaf_samples = min_leaf_samples
        self._node = {
                        'left': None,
                        'right': None,
                        'feature': None,
                        'threshold': None,
                        'depth': 0,
                        'pred': None
                    }
        self.tree = None  # словарь в котором будет храниться построенное дерево

    def fit(self, X, y): 
        self.tree = {'root': self._node.copy()}  # создаём первую узел в дереве
        self._build_tree(self.tree['root'], X, y)  # запускаем рекурсивную функцию для построения дерева
        return self

    def predict(self, X):
        preds = []
        for x in X:
            preds_for_x = self._get_predict(self.tree['root'], x)  # рекурсивно ищем лист в дереве соответствующий объекту
            preds.append(preds_for_x)
        preds = np.array(preds)
        return preds[:, None]

    def get_best_split(self, X, y):

        best_Q = None
        best_j = None
        best_t = None
        best_left_ids = None
        best_right_ids = None
        
        full_mse = self.mse(y)
        features = self.get_random_features_inds(X.shape[1])
        for j in features:
            split_values = self.find_splits(X, j)
            for t in split_values:
                left_mask = X[:, j] <= t
                right_mask = X[:, j] > t
                left_y, right_y = y[left_mask], y[right_mask]
                Q_cur = self.calc_Q(left_y, right_y, full_mse)
                if best_Q is None or Q_cur > best_Q:
                    best_Q = Q_cur
                    best_j = j
                    best_t = t
                    best_left_ids = left_mask
                    best_right_ids = right_mask

        return best_j, best_t, best_left_ids, best_right_ids

    def calc_Q(self, y_left, y_right, full_mse):
        n_l = len(y_left)
        n_r = len(y_right)
        n = n_l + n_r
        return full_mse - (n_l*self.mse(y_left) + n_r*self.mse(y_right))/n
    
    def mse(self, y):
        n = len(y)
        if n == 0:
            return 0
        y_mean = np.mean(y)
        return np.sum((y - y_mean)**2)/n
    
    def find_splits(self, X, column):
        X_unique = np.unique(X[:, column])
        split_values = np.empty(X_unique.shape[0] - 1)
        for i in range(1, X_unique.shape[0]):
            average = (X_unique[i - 1] + X_unique[i]) / 2
            split_values[i - 1] = average
        return split_values
    
    def get_random_features_inds(self, n):
        all_inds = np.arange(n)
        random.shuffle(all_inds)
        return all_inds[:self.max_features]
        

    def _build_tree(self, curr_node, X, y):

        if curr_node['depth'] == self.max_depth:  # выход из рекурсии если построили до максимальной глубины
            # сохраняем предсказания листьев дерева перед выходом из рекурсии
            curr_node['pred'] = y.mean()
            return

        if len(y) < 2:  # выход из рекурсии значения если "y" одинковы для все объектов
            curr_node['pred'] = y.mean()
            return

        j, t, left_ids, right_ids = self.get_best_split(X, y)  # нахождение лучшего разбиения

        curr_node['feature'] = j  # признак по которому производится разбиение в текущем узле
        curr_node['threshold'] = t  # порог по которому производится разбиение в текущем узле

        left = self._node.copy()  # создаём узел для левого поддерева
        right = self._node.copy()  # создаём узел для правого поддерева

        left['depth'] = curr_node['depth'] + 1  # увеличиваем значение глубины в узлах поддеревьев
        right['depth'] = curr_node['depth'] + 1

        curr_node['left'] = left
        curr_node['right'] = right
        curr_node['pred'] = y.mean()

        self._build_tree(left, X[left_ids], y[left_ids])  # продолжаем построение дерева
        self._build_tree(right, X[right_ids], y[right_ids])

    def _get_predict(self, node, x):
        if node['threshold'] is None:  # если в узле нет порога, значит это лист, выходим из рекурсии
            return node['pred']

        if x[node['feature']] <= node['threshold']:  # уходим в правое или левое поддерево в зависимости от порога и признака
            return self._get_predict(node['left'], x)
        else:
            return self._get_predict(node['right'], x)

    
    
class MyGradientBoostingRegressor:
    
    def __init__(self, n_estimators=50, max_depth=5, max_features=3, learing_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learing_rate
        self.max_depth = max_depth
        self.max_features = max_features 
        self.estimators = []
        
    def fit(self, X, y):
        
        res = y.copy()
        """You code here"""
        for _ in tqdm(range(self.n_estimators)):
            gb = MyDecisionTreeRegressor(max_depth=self.max_depth, max_features=self.max_features)
            gb.fit(X, res)
            res = res - self.learning_rate*gb.predict(X)
            self.estimators.append(gb)
        
        return self
    
    def predict(self, X):
        
        preds = np.array([gb.predict(X)*self.learning_rate for gb in self.estimators])
        predict = preds.sum(axis=0)
        
        return predict

In [423]:
gbm = MyGradientBoostingRegressor(max_depth=5, max_features=3)
gbm.fit(X=X_train_boston, y=y_train_boston)
(abs(y_test_boston - gbm.predict(X_test_boston)) / y_test_boston).mean()

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:25<00:00,  1.98it/s]


0.4039167330716401

In [415]:
def read_matrix(n, dtype=float):
    matrix = np.array([list(map(dtype, input().split())) for _ in range(n)])
    return matrix

def read_input_matriсes(n, m, k):
    X_train, y_train, X_test = read_matrix(n), read_matrix(n), read_matrix(k)
    return X_train, y_train, X_test

def print_matrix(matrix):
    for row in matrix:
        print(' '.join(map(str, row)))
        
def solution():
    n, m, k = 3, 4, 2
    X_train = np.array([[1.0, 0.0, 0.0, 0.0],
                        [2.0, 0.0, 0.0, 0.0],
                        [3.0, 0.0, 0.0, 0.0]]) 
    y_train =  np.array([[0],
                        [1],
                        [2]]) 
    X_test = np.array([[1.0, 0.0, 0.0, 0.0],
                       [2.0, 0.0, 0.0, 0.0]])

    gb = MyGradientBoostingRegressor(max_depth=2, max_features=4)  
    gb.fit(X_train, y_train)

    predictions = gb.predict(X_test)
    print(predictions)
    print_matrix(predictions)

solution()

100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 1671.27it/s]

[[0.        ]
 [0.87842335]]
0.0
0.8784233454094308
